# Ensemble Techniques

## Environment Setup

### Imports

In [1]:
import os
os.sys.path.append("utils")

import torch
import numpy as np
import pandas as pd
from functools import reduce
import torch.nn.functional as F
from monai.transforms import AsDiscrete
from utils.Inference import ensemble_inference

### Config

In [2]:
seed = 33
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)

## Load Test Data

In [3]:
subject_ids = pd.read_csv('./data/TEST.csv')['SubjectID'].values

ah_segs, unet_segs, segresnet_segs, untr_segs, gt_segs = [], [], [], [], []
for sid in subject_ids:
    ah_channels, unet_channels, segresnet_channels, untr_channels, gt_channels = [], [], [], [], []
    for channel in ['TC', 'WT', 'ET']:
        ah_channels.append(f'./outputs/AHNet/pred_segs/test_pred_segs/pred_{sid}_{channel}.npz')
        unet_channels.append(f'./outputs/UNet/pred_segs/test_pred_segs/pred_{sid}_{channel}.npz')
        segresnet_channels.append(f'./outputs/SegResNet/pred_segs/test_pred_segs/pred_{sid}_{channel}.npz')
        untr_channels.append(f'./outputs/UNETR/pred_segs/test_pred_segs/pred_{sid}_{channel}.npz')
        gt_channels.append(f'./outputs/gt_segs/test_gt_segs/gt_{sid}_{channel}.npz')
    
    ah_segs.append(ah_channels)
    unet_segs.append(unet_channels)
    segresnet_segs.append(segresnet_channels)
    untr_segs.append(untr_channels)
    gt_segs.append(gt_channels)

# Dataframe
test_df = pd.DataFrame()
test_df['SubjectID'] = subject_ids
test_df['AHNet'] = ah_segs
test_df['UNet'] = unet_segs
test_df['SegResNet'] = segresnet_segs
test_df['UNETR'] = untr_segs
test_df['GT'] = gt_segs

test_df.head()

,SubjectID,AHNet,UNet,SegResNet,UNETR,GT
0,100214B,"[./outputs/AHNet/pred_segs/test_pred_segs/pred_100214B_TC.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100214B_WT.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100214B_ET.npz]","[./outputs/UNet/pred_segs/test_pred_segs/pred_100214B_TC.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100214B_WT.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100214B_ET.npz]","[./outputs/SegResNet/pred_segs/test_pred_segs/pred_100214B_TC.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100214B_WT.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100214B_ET.npz]","[./outputs/UNETR/pred_segs/test_pred_segs/pred_100214B_TC.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100214B_WT.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100214B_ET.npz]","[./outputs/gt_segs/test_gt_segs/gt_100214B_TC.npz, ./outputs/gt_segs/test_gt_segs/gt_100214B_WT.npz, ./outputs/gt_segs/test_gt_segs/gt_100214B_ET.npz]"
1,100340A,"[./outputs/AHNet/pred_segs/test_pred_segs/pred_100340A_TC.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100340A_WT.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100340A_ET.npz]","[./outputs/UNet/pred_segs/test_pred_segs/pred_100340A_TC.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100340A_WT.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100340A_ET.npz]","[./outputs/SegResNet/pred_segs/test_pred_segs/pred_100340A_TC.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100340A_WT.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100340A_ET.npz]","[./outputs/UNETR/pred_segs/test_pred_segs/pred_100340A_TC.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100340A_WT.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100340A_ET.npz]","[./outputs/gt_segs/test_gt_segs/gt_100340A_TC.npz, ./outputs/gt_segs/test_gt_segs/gt_100340A_WT.npz, ./outputs/gt_segs/test_gt_segs/gt_100340A_ET.npz]"
2,100391A,"[./outputs/AHNet/pred_segs/test_pred_segs/pred_100391A_TC.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100391A_WT.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100391A_ET.npz]","[./outputs/UNet/pred_segs/test_pred_segs/pred_100391A_TC.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100391A_WT.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100391A_ET.npz]","[./outputs/SegResNet/pred_segs/test_pred_segs/pred_100391A_TC.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100391A_WT.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100391A_ET.npz]","[./outputs/UNETR/pred_segs/test_pred_segs/pred_100391A_TC.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100391A_WT.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100391A_ET.npz]","[./outputs/gt_segs/test_gt_segs/gt_100391A_TC.npz, ./outputs/gt_segs/test_gt_segs/gt_100391A_WT.npz, ./outputs/gt_segs/test_gt_segs/gt_100391A_ET.npz]"
3,100190B,"[./outputs/AHNet/pred_segs/test_pred_segs/pred_100190B_TC.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100190B_WT.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100190B_ET.npz]","[./outputs/UNet/pred_segs/test_pred_segs/pred_100190B_TC.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100190B_WT.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100190B_ET.npz]","[./outputs/SegResNet/pred_segs/test_pred_segs/pred_100190B_TC.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100190B_WT.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100190B_ET.npz]","[./outputs/UNETR/pred_segs/test_pred_segs/pred_100190B_TC.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100190B_WT.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100190B_ET.npz]","[./outputs/gt_segs/test_gt_segs/gt_100190B_TC.npz, ./outputs/gt_segs/test_gt_segs/gt_100190B_WT.npz, ./outputs/gt_segs/test_gt_segs/gt_100190B_ET.npz]"
4,100142A,"[./outputs/AHNet/pred_segs/test_pred_segs/pred_100142A_TC.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100142A_WT.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100142A_ET.npz]","[./outputs/UNet/pred_segs/test_pred_segs/pred_1

## Ensemble 1 - Average

In [4]:
def avg(predictions):
    stacked_predictions = torch.stack(predictions, dim=0)
    return torch.mean(stacked_predictions, dim=0)

#---------------------------------------------------------#

best_mean = 0
best_threshold = 0
df = None
for threshold in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    avg_df = ensemble_inference(test_df, avg, threshold)
    mean = avg_df['Dice'].mean()
    print(f'Average Mean Dice for threshold {threshold}: {mean}')
    if mean > best_mean:
        best_mean = mean
        best_threshold = threshold
        df = avg_df
print(f'BEST THRESHOLD: {best_threshold}')
df.to_csv('./outputs/Ensemble/ensemble_avg.csv', index=False)
df.describe()

Average Mean Dice for threshold 0.1: 0.5150403138250113
Average Mean Dice for threshold 0.2: 0.5272680526043498
Average Mean Dice for threshold 0.3: 0.6201561954953978
Average Mean Dice for threshold 0.4: 0.6306360651919197
Average Mean Dice for threshold 0.5: 0.6117088165014021
Average Mean Dice for threshold 0.6: 0.582749746499523
Average Mean Dice for threshold 0.7: 0.5514685651587863
Average Mean Dice for threshold 0.8: 0.4944665648283497
Average Mean Dice for threshold 0.9: 0.44803049511486487
BEST THRESHOLD: 0.4


,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.630636,0.662461,0.673749,0.533071,6.677419,6.516129,87.129032,8.709677,8.709677,5.129032,16592.322581,16591.774194,14575.709677,17105.387097,17985.870968,13810.806452
std,0.259768,0.255310,0.260801,0.293511,5.906985,5.977017,166.985177,9.103822,9.103822,4.951376,24168.923954,24207.681579,22133.776319,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,7.000000,6.000000,0.000000,44.000000,44.000000,0.000000
25%,0.484863,0.554968,0.560128,0.282297,3.000000,2.500000,5.500000,2.000000,2.000000,1.500000,1057.000000,1048.000000,662.500000,802.000000,802.000000,410.000000
50%,0.676840,0.717632,0.698113,0.542388,5.000000,5.000000,14.000000,6.000000,6.000000,4.000000,4993.000000,4986.000000,3298.000000,3847.000000,4138.000000,1944.000000
75%,0.842653,0.869613,0.885937,0.769723,8.000000,8.000000,65.000000,13.500000,13.500000,7.000000,29342.500000,29308.000000,27872.000000,26305.000000,27036.500000,23520.500000
max,0.946255,0.955846,0.958104,0.924816,25.000000,25.000000,684.000000,43.000000,43.000000,21.000000,90746.000000,90706.000000,83437.000000,102770.000000,105729.000000,92221.000000


## Ensemble 2 - Weighted Average (Mean Dice Weights)

In [5]:
def weigthed_avg(predictions):

    # Weights form validation set AHNet, SegResNet, UNet, UNETR
    weights = np.array([0.546402, 0.648790, 0.428639, 0.547957])
    weights = weights / np.sum(weights)
    weights = torch.tensor(weights).float()
    # Weighted average
    stacked_predictions = torch.stack(predictions, dim=0)
    return torch.sum(stacked_predictions * weights[:, None, None, None, None, None], dim=0)
     
#---------------------------------------------------------#

best_mean = 0
best_threshold = 0
df = None
for threshold in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    weigthed_avg_df = ensemble_inference(test_df, weigthed_avg, threshold)
    mean = weigthed_avg_df['Dice'].mean()
    print(f'Weighted Average Mean Dice for threshold {threshold}: {mean}')
    if mean > best_mean:
        best_mean = mean
        best_threshold = threshold
        df = weigthed_avg_df
print(f'BEST THRESHOLD: {best_threshold}')
df.to_csv('./outputs/Ensemble/ensemble_weighted_avg.csv', index=False)
df.describe()

Weighted Average Mean Dice for threshold 0.1: 0.5145711503442256
Weighted Average Mean Dice for threshold 0.2: 0.576161464856517
Weighted Average Mean Dice for threshold 0.3: 0.632053408891924
Weighted Average Mean Dice for threshold 0.4: 0.6359004551965383
Weighted Average Mean Dice for threshold 0.5: 0.6326853892495555
Weighted Average Mean Dice for threshold 0.6: 0.5931650337673002
Weighted Average Mean Dice for threshold 0.7: 0.5561348192634121
Weighted Average Mean Dice for threshold 0.8: 0.49360962188051594
Weighted Average Mean Dice for threshold 0.9: 0.4448244151088499
BEST THRESHOLD: 0.4


,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.635900,0.667215,0.679828,0.538870,6.903226,6.419355,109.774194,8.709677,8.709677,5.129032,16629.322581,16633.903226,14432.903226,17105.387097,17985.870968,13810.806452
std,0.257415,0.252058,0.256418,0.295345,5.985287,5.863868,204.315884,9.103822,9.103822,4.951376,24313.317312,24377.260244,22128.085428,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,6.000000,5.000000,0.000000,44.000000,44.000000,0.000000
25%,0.498647,0.566927,0.570827,0.290649,2.500000,2.000000,6.000000,2.000000,2.000000,1.500000,1042.000000,1032.500000,635.000000,802.000000,802.000000,410.000000
50%,0.675439,0.723906,0.727524,0.549588,5.000000,5.000000,21.000000,6.000000,6.000000,4.000000,4940.000000,4945.000000,3194.000000,3847.000000,4138.000000,1944.000000
75%,0.846335,0.871845,0.890754,0.776012,8.500000,8.000000,103.500000,13.500000,13.500000,7.000000,29476.500000,29444.500000,27817.000000,26305.000000,27036.500000,23520.500000
max,0.947493,0.956289,0.958427,0.927762,25.000000,25.000000,922.000000,43.000000,43.000000,21.000000,91540.000000,91606.000000,83508.000000,102770.000000,105729.000000,92221.000000


## Ensemble 3 - Median 

In [6]:
def median(predictions):
    stacked_predictions = torch.stack(predictions, dim=0)
    return torch.median(stacked_predictions, dim=0).values

#---------------------------------------------------------#

best_mean = 0
best_threshold = 0
df = None
for threshold in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    median_df = ensemble_inference(test_df, median, threshold)
    mean = median_df['Dice'].mean()
    print(f'Median Mean Dice for threshold {threshold}: {mean}')
    if mean > best_mean:
        best_mean = mean
        best_threshold = threshold
        df = median_df
print(f'BEST THRESHOLD: {best_threshold}')
df.to_csv('./outputs/Ensemble/ensemble_median.csv', index=False)
df.describe()

Median Mean Dice for threshold 0.1: 0.6100782444400172
Median Mean Dice for threshold 0.2: 0.6120831245376218
Median Mean Dice for threshold 0.3: 0.6029955754356999
Median Mean Dice for threshold 0.4: 0.592251036436327
Median Mean Dice for threshold 0.5: 0.5806126330168017
Median Mean Dice for threshold 0.6: 0.5694510761287904
Median Mean Dice for threshold 0.7: 0.5551219413117054
Median Mean Dice for threshold 0.8: 0.536392334788557
Median Mean Dice for threshold 0.9: 0.5016900248104527
BEST THRESHOLD: 0.2


,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.612083,0.638850,0.649940,0.531447,6.225806,5.741935,7.032258,8.709677,8.709677,5.129032,15876.483871,16000.548387,13569.354839,17105.387097,17985.870968,13810.806452
std,0.273661,0.269515,0.274134,0.306581,5.948163,5.483112,8.121920,9.103822,9.103822,4.951376,24037.063603,24258.577521,21550.894202,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.486434,0.508295,0.520046,0.242854,2.500000,2.000000,2.000000,2.000000,2.000000,1.500000,943.500000,937.500000,592.000000,802.000000,802.000000,410.000000
50%,0.635331,0.679783,0.720000,0.571091,4.000000,4.000000,4.000000,6.000000,6.000000,4.000000,3859.000000,3888.000000,2995.000000,3847.000000,4138.000000,1944.000000
75%,0.850039,0.868642,0.884660,0.785967,8.000000,7.500000,9.000000,13.500000,13.500000,7.000000,27416.000000,27688.000000,25957.500000,26305.000000,27036.500000,23520.500000
max,0.947509,0.954614,0.956809,0.932726,25.000000,25.000000,35.000000,43.000000,43.000000,21.000000,90012.000000,90697.000000,81341.000000,102770.000000,105729.000000,92221.000000


## Ensemble 4 - Maximum Confidence 

In [7]:
def max_confidence(predictions):
    stacked_predictions = torch.stack(predictions, dim=0)
    return torch.max(stacked_predictions, dim=0).values

#---------------------------------------------------------#

best_mean = 0
best_threshold = 0
df = None
for threshold in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    max_confidence_df = ensemble_inference(test_df, max_confidence, threshold)
    mean = max_confidence_df['Dice'].mean()
    print(f'Max Confidence Mean Dice for threshold {threshold}: {mean}')
    if mean > best_mean:
        best_mean = mean
        best_threshold = threshold
        df = max_confidence_df
print(f'BEST THRESHOLD: {best_threshold}')
df.to_csv('./outputs/Ensemble/ensemble_max_confidence.csv', index=False)
df.describe()

Max Confidence Mean Dice for threshold 0.1: 0.4769555576325905
Max Confidence Mean Dice for threshold 0.2: 0.5018590381669421
Max Confidence Mean Dice for threshold 0.3: 0.5132624893899886
Max Confidence Mean Dice for threshold 0.4: 0.5196868994032904
Max Confidence Mean Dice for threshold 0.5: 0.5234780021372342
Max Confidence Mean Dice for threshold 0.6: 0.5252501968624851
Max Confidence Mean Dice for threshold 0.7: 0.5257121397167324
Max Confidence Mean Dice for threshold 0.8: 0.5242511283835366
Max Confidence Mean Dice for threshold 0.9: 0.5196892056584118
BEST THRESHOLD: 0.7


,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.525712,0.563654,0.575627,0.437288,25.709677,21.967742,37.870968,8.709677,8.709677,5.129032,19678.903226,19567.967742,19298.677419,17105.387097,17985.870968,13810.806452
std,0.318874,0.324253,0.333119,0.309278,43.836205,36.408318,55.675094,9.103822,9.103822,4.951376,25807.250693,25731.658040,25660.328382,25565.209783,26912.616906,22937.645993
min,0.002576,0.005687,0.002040,0.000000,1.000000,2.000000,1.000000,1.000000,1.000000,0.000000,1821.000000,1813.000000,1750.000000,44.000000,44.000000,0.000000
25%,0.218996,0.249058,0.249256,0.133651,8.500000,7.000000,10.000000,2.000000,2.000000,1.500000,3700.500000,3664.000000,3465.500000,802.000000,802.000000,410.000000
50%,0.632381,0.661653,0.712883,0.479532,13.000000,11.000000,19.000000,6.000000,6.000000,4.000000,6310.000000,5889.000000,5875.000000,3847.000000,4138.000000,1944.000000
75%,0.794857,0.850547,0.873580,0.694644,22.000000,19.500000,30.000000,13.500000,13.500000,7.000000,32187.000000,32010.000000,32388.000000,26305.000000,27036.500000,23520.500000
max,0.932704,0.950421,0.953007,0.894683,221.000000,190.000000,251.000000,43.000000,43.000000,21.000000,99301.000000,98700.000000,97619.000000,102770.000000,105729.000000,92221.000000


## Ensemble 5 - Majority Voting

In [8]:
def majority_voting(predictions):
    transforms =[AsDiscrete(threshold=0.3), AsDiscrete(threshold=0.5), AsDiscrete(threshold=0.2), AsDiscrete(threshold=0.4)]
    predictions = [transform(prediction) for transform, prediction in zip(transforms, predictions)]
    stacked_predictions = torch.stack(predictions, dim=0)  
    return torch.mode(stacked_predictions, dim=0).values

#---------------------------------------------------------#

majority_vote_df = ensemble_inference(test_df, majority_voting)
majority_vote_df.to_csv('./outputs/Ensemble/ensemble_majority_vote.csv', index=False)
majority_vote_df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.598177,0.629584,0.634287,0.518907,5.774194,5.322581,6.967742,8.709677,8.709677,5.129032,14518.096774,14620.096774,12023.290323,17105.387097,17985.870968,13810.806452
std,0.281068,0.273253,0.281470,0.319628,5.529977,5.042401,8.130125,9.103822,9.103822,4.951376,22096.846216,22308.181914,19524.494051,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.406790,0.476757,0.476810,0.194828,2.000000,2.000000,2.000000,2.000000,2.000000,1.500000,735.000000,726.500000,461.000000,802.000000,802.000000,410.000000
50%,0.654574,0.685359,0.715392,0.594832,4.000000,4.000000,6.000000,6.000000,6.000000,4.000000,3544.000000,3562.000000,2571.000000,3847.000000,4138.000000,1944.000000
75%,0.832495,0.866259,0.871473,0.795037,7.500000,7.000000,8.500000,13.500000,13.500000,7.000000,24593.000000,24801.500000,21853.000000,26305.000000,27036.500000,23520.500000
max,0.951305,0.957194,0.959260,0.937462,21.000000,21.000000,38.000000,43.000000,43.000000,21.000000,82128.000000,82342.000000,73322.000000,102770.000000,105729.000000,92221.000000


## Ensemble 6 - Dempster-Shafer Theory

In [4]:
def dempster_rule(mass1, mass2):

    K = 1 / (1 - mass1[..., 0] * mass2[..., 1] - mass1[..., 1] * mass2[..., 0])

    combined_mass = torch.zeros_like(mass1)
    combined_mass[..., 0] = K * (mass1[..., 0] * mass2[..., 0] + mass1[..., 0] * mass2[..., 2] + mass1[..., 2] * mass2[..., 0])
    combined_mass[..., 1] = K * (mass1[..., 1] * mass2[..., 1] + mass1[..., 1] * mass2[..., 2] + mass1[..., 2] * mass2[..., 1])
    combined_mass[..., 2] = K * (mass1[..., 0] * mass2[..., 1] + mass1[..., 1] * mass2[..., 0] + mass1[..., 2] * mass2[..., 2])
    
    return combined_mass

def dempster_shafer_inference(predictions):

    # Params
    predictions = [pred.squeeze(0) for pred in predictions]

    # Mass Beliefs for {tumor, non-tumor} - (3, 3, x, y, z) - (masses, channels, x, y, z)
    masses = []	
    for pred in predictions:
        epsilon = 1 - torch.max(pred) 
        tumor_mass = pred
        non_tumor_mass = 1 - pred
        uncertainty_mass = torch.full_like(pred, epsilon)
        masses.append(torch.stack([tumor_mass, non_tumor_mass, uncertainty_mass], dim=-1))

    combined_mass = reduce(dempster_rule, masses)

    # Normalized Mass
    combined_mass_sum = combined_mass.sum(dim=-1, keepdim=True)
    normalized_mass = combined_mass / combined_mass_sum
    
    return normalized_mass[..., 0].unsqueeze(0)   

#---------------------------------------------------------#

ds_df = ensemble_inference(test_df, dempster_shafer_inference, threshold=0.1, store_npz=True, model_name='ds')
ds_df.to_csv('./outputs/Ensemble/ensemble_dempster_shafer.csv', index=False)
ds_df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.577118,0.606080,0.618176,0.490236,6.290323,6.225806,6.741935,8.709677,8.709677,5.129032,15446.258065,15558.225806,13528.290323,17105.387097,17985.870968,13810.806452
std,0.279771,0.276253,0.281809,0.313583,5.768845,5.649246,6.093536,9.103822,9.103822,4.951376,23810.414574,23963.213483,21562.279380,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,16.000000,12.000000,0.000000,44.000000,44.000000,0.000000
25%,0.442785,0.495962,0.520909,0.213849,4.000000,4.000000,3.000000,2.000000,2.000000,1.500000,698.000000,724.000000,533.500000,802.000000,802.000000,410.000000
50%,0.565420,0.608258,0.612368,0.492505,5.000000,5.000000,5.000000,6.000000,6.000000,4.000000,3781.000000,3802.000000,3008.000000,3847.000000,4138.000000,1944.000000
75%,0.834014,0.862115,0.876348,0.775048,7.000000,7.000000,8.500000,13.500000,13.500000,7.000000,27914.500000,28144.500000,26598.000000,26305.000000,27036.500000,23520.500000
max,0.948727,0.954471,0.957102,0.934608,28.000000,28.000000,27.000000,43.000000,43.000000,21.000000,89020.000000,89231.000000,81336.000000,102770.000000,105729.000000,92221.000000


## Ensemble 7 - CRF - No Energy Minimization

In [11]:
def crf_ne(predictions, sigma_spatial=3.0, sigma_bilateral=1.0, iterations=10, alpha=0.7):
    # Weighted Average
    weights = torch.tensor([0.546402, 0.648790, 0.428639, 0.547957]).float()
    weights = weights / weights.sum()
    stacked_preds = torch.stack(predictions, dim=0)
    avg_pred = torch.sum(stacked_preds * weights[:, None, None, None, None, None], dim=0)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    avg_pred = avg_pred.to(device)
    Q = avg_pred.clone()

    for _ in range(iterations):
        # Spatial
        Q_spatial = F.avg_pool3d(Q, kernel_size=3, stride=1, padding=1)

        # Bilateral
        diff = (Q.unsqueeze(2) - Q.unsqueeze(1)).pow(2)
        bilateral_weight = torch.exp(-diff / (2 * sigma_bilateral**2))
        Q_bilateral = (Q.unsqueeze(2) * bilateral_weight).sum(1) / bilateral_weight.sum(1)

        # Combine spatial and bilateral
        Q_new = Q * torch.exp((Q_spatial - Q) / sigma_spatial + (Q_bilateral - Q) / sigma_bilateral)
        Q_new = Q_new / Q_new.sum(dim=1, keepdim=True)

        Q = Q_new
        
    final_pred = alpha * avg_pred + (1 - alpha) * Q

    return final_pred.cpu()

#--------------------------------------------------------------------------------

df = ensemble_inference(test_df, crf_ne, threshold=0.4, store_npz=True, model_name='crf_ne')
df.to_csv('./outputs/Ensemble/ensemble_crf_ne.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.635137,0.666215,0.678172,0.538820,6.645161,6.548387,123.709677,8.709677,8.709677,5.129032,16324.032258,16321.354839,14036.870968,17105.387097,17985.870968,13810.806452
std,0.259541,0.253901,0.258408,0.298055,5.689454,6.048905,232.605129,9.103822,9.103822,4.951376,23963.957376,24025.909997,21674.041599,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,5.000000,2.000000,0.000000,44.000000,44.000000,0.000000
25%,0.500498,0.559208,0.571193,0.293311,3.500000,2.000000,7.000000,2.000000,2.000000,1.500000,992.000000,974.000000,584.000000,802.000000,802.000000,410.000000
50%,0.688387,0.722104,0.740951,0.561484,5.000000,5.000000,26.000000,6.000000,6.000000,4.000000,4791.000000,4748.000000,3074.000000,3847.000000,4138.000000,1944.000000
75%,0.846332,0.870643,0.889466,0.781494,8.500000,8.000000,126.500000,13.500000,13.500000,7.000000,29037.000000,29010.000000,27191.000000,26305.000000,27036.500000,23520.500000
max,0.948949,0.957131,0.959748,0.929968,25.000000,25.000000,1103.000000,43.000000,43.000000,21.000000,90066.000000,90116.000000,81821.000000,102770.000000,105729.000000,92221.000000


## Ensemble 8 - CRF - With Energy Minimization

In [4]:
def crf(predictions, iterations=5, sigma_spatial=3.0, sigma_bilateral=1.0, learning_rate=1e-3, alpha=0.7):
    # Weighted Average
    weights = torch.tensor([0.546402, 0.648790, 0.428639, 0.547957]).float()
    weights = weights / weights.sum()
    stacked_preds = torch.stack(predictions, dim=0)
    avg_pred = torch.sum(stacked_preds * weights[:, None, None, None, None, None], dim=0)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    avg_pred = avg_pred.to(device)

    Q = avg_pred.clone().requires_grad_(True)

    for _ in range(iterations):
        # Compute energy
        unary_energy = -torch.sum(avg_pred * torch.log(Q + 1e-10))
        
        # Spatial pairwise energy
        Q_spatial = F.avg_pool3d(Q, kernel_size=3, stride=1, padding=1)
        spatial_energy = torch.sum((Q - Q_spatial).pow(2)) / (2 * sigma_spatial**2)
        
        # Bilateral pairwise energy
        diff = (Q.unsqueeze(2) - Q.unsqueeze(1)).pow(2)
        bilateral_weight = torch.exp(-diff / (2 * sigma_bilateral**2))
        Q_bilateral = (Q.unsqueeze(2) * bilateral_weight).sum(1) / bilateral_weight.sum(1)
        bilateral_energy = torch.sum((Q - Q_bilateral).pow(2)) / (2 * sigma_bilateral**2)
        
        # Total energy
        energy = unary_energy + spatial_energy + bilateral_energy
        
        # Update Q
        energy.backward()
        with torch.no_grad():
            Q -= learning_rate * Q.grad
            Q.grad.zero_()
        Q = F.softmax(Q, dim=1)
        Q = Q.detach().requires_grad_(True)

    final_pred = alpha * avg_pred + (1 - alpha) * Q
    
    return final_pred.detach().cpu()

#--------------------------------------------------------------------------------

df = ensemble_inference(test_df, crf, threshold=0.4, store_npz=True, model_name='crf')
df.to_csv('./outputs/Ensemble/ensemble_crf.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.635099,0.666059,0.678025,0.538904,6.612903,6.387097,123.451613,8.709677,8.709677,5.129032,16321.967742,16317.000000,14038.354839,17105.387097,17985.870968,13810.806452
std,0.259750,0.253963,0.259043,0.297947,5.643152,6.059029,231.764512,9.103822,9.103822,4.951376,23962.280433,24021.201451,21676.087175,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,5.000000,2.000000,0.000000,44.000000,44.000000,0.000000
25%,0.500684,0.558539,0.571357,0.293646,3.500000,2.000000,7.000000,2.000000,2.000000,1.500000,992.500000,973.500000,583.500000,802.000000,802.000000,410.000000
50%,0.691589,0.722104,0.740583,0.561484,5.000000,5.000000,26.000000,6.000000,6.000000,4.000000,4791.000000,4746.000000,3075.000000,3847.000000,4138.000000,1944.000000
75%,0.846302,0.870695,0.889422,0.781489,8.000000,8.000000,125.500000,13.500000,13.500000,7.000000,29038.500000,29009.000000,27190.500000,26305.000000,27036.500000,23520.500000
max,0.948939,0.957131,0.959761,0.929925,25.000000,25.000000,1097.000000,43.000000,43.000000,21.000000,90051.000000,90081.000000,81829.000000,102770.000000,105729.000000,92221.000000
